# 01 - Interview Question Generation
`Author: Abdlazeez Jimoh`

In [11]:
import json
from typing import Optional

from openai import OpenAI

from lib.agents import BaseAgent
from lib.configs import OPENAI_API_KEY
from lib.types import Question


class OpenAIQuestionGeneratorAgent(BaseAgent):
    def __init__(self):
        super().__init__()
        self.client = OpenAI(api_key=OPENAI_API_KEY)
        self.system_prompt = """You are a non-technical interviewer that interviews \
across the following categories:
- personal
- role-specific
- behavioural
- situational

You will be provided with a candidate's description.

Generate {n_questions} questions, ensuring that there is a question for each category \
and the questions should be based on the candidate's description.

* You answer strictly as a list of JSON objects. Don't include any other verbose texts, \
and don't include the markdown syntax anywhere.

JSON format:
[
    {{"question": "<personal_question>", "type": "personal"}},
    {{"question": "<role_specific_question>", "type": "role-specific"}},
    {{"question": "<behavioural_question>", "type": "behavioural"}},
    {{"question": "<situational_question>", "type": "situational"}},
    ...more questions to make up {n_questions} questions
]"""

        self.user_prompt = "Candidate Description:\n{description}"

    def __call__(self, description: str, n_questions: int = 4) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def run(self, description: str, n_questions: int = 4) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int, optional): The number of questions to generate. Defaults to 4.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        # Generate questions
        questions = self._generate(description, n_questions)

        return questions

    def _generate(self, description: str, n_questions: int) -> Optional[list[Question]]:
        """
        Generate interview questions based on the given description.

        Args:
            description (str): The description used as input for question generation.
            n_questions (int): The number of questions to generate.

        Returns:
            Optional[list[Question]]: A list of generated interview questions or None if an error occurs.
        """

        try:
            # Ensure that there are at least 4 questions
            if n_questions < 4:
                n_questions = 4

            output = self.client.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                messages=[
                    {
                        "role": "system",
                        "content": self.system_prompt.format(n_questions=n_questions),
                    },
                    {
                        "role": "user",
                        "content": self.user_prompt.format(description=description),
                    },
                ],
                temperature=0.5,
                max_tokens=1024,
                top_p=1,
                frequency_penalty=0,
                presence_penalty=0,
            )
            questions = json.loads(output.choices[0].message.content or "[]")

            return questions
        except Exception as e:
            return None

In [8]:
from pprint import pprint


question_generator = OpenAIQuestionGeneratorAgent()
questions = question_generator.run(
    "a software engineer at a startup in San Francisco. I have 5 years of experience and I'm looking for a new job.",
    5
)

pprint(questions)

[{'question': 'What motivated you to pursue a career in software engineering?',
  'type': 'personal'},
 {'question': 'Can you describe a challenging project you worked on at your '
              'current startup and how you overcame any obstacles?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time when you had to work with a difficult team '
              'member. How did you handle the situation?',
  'type': 'behavioural'},
 {'question': 'You have a tight deadline to deliver a new feature, but the '
              'requirements keep changing. How would you approach this '
              'situation?',
  'type': 'situational'},
 {'question': 'How do you prioritize your tasks when working on multiple '
              'projects simultaneously?',
  'type': 'role-specific'}]


In [12]:
questions = question_generator.run(
    "a data scientist from India. I have 3 years of experience. I've worked on a variety of projects, including a recommendation engine for a large e-commerce company. I've led a team of 5 data scientists and engineers.",
    8
)

pprint(questions)

[{'question': 'What motivated you to pursue a career in data science?',
  'type': 'personal'},
 {'question': 'Can you describe a specific project where you implemented a '
              'recommendation engine?',
  'type': 'role-specific'},
 {'question': 'Tell me about a time when you had to lead a team towards a '
              'common goal. How did you approach it?',
  'type': 'behavioural'},
 {'question': 'How would you handle a situation where a project deadline is '
              'approaching, and the team is facing technical challenges?',
  'type': 'situational'},
 {'question': 'What do you enjoy most about working on data science projects?',
  'type': 'personal'},
 {'question': 'What techniques or algorithms have you found most effective '
              'when working on recommendation engines?',
  'type': 'role-specific'},
 {'question': 'Can you provide an example of a difficult decision you had to '
              'make as a leader of a data science team?',
  'type': 'behavioural